# Creando conexiones y extrayendo datos desde SQL server con PYODBC y SQLALCHEMY

En este proyecto, lo que haremos es conectar Python con una base de datos SQL Server utilizando bibliotecas como pyodbc y sqlalchemy. Esto nos permitirá acceder y extraer datos de forma eficiente, manejándolos cómodamente en pandas. Es útil porque simplifica la interacción con bases de datos y automatiza procesos de consulta. Además, es clave tener en cuenta la configuración correcta de las credenciales y el controlador para evitar problemas de conexión. ¡Con estas herramientas, trabajar con SQL Server será mucho más sencillo y organizado!

## Importamos las librerias necesarias

In [1]:
!pip install pyodbc

import sqlalchemy
import pandas as pd
import pyodbc as podbc

## Conexión con PYODBC

In [2]:
import pandas as pd
import pyodbc as podbc

def conexion_sql_server():
    """
    Establece una conexión con una base de datos SQL Server usando ODBC.

    Este método utiliza el controlador ODBC 17 para SQL Server y se conecta a
    la instancia de SQL Server especificada.

    Devuelve:
        connection: Un objeto de conexión a la base de datos de SQL Server.
        
    """
    connection = podbc.connect(
        Trusted_Connection='yes',
        Driver='{ODBC Driver 17 for SQL Server}',
        Server='DESKTOP-PTELUGB\SQLEXPRESS',
        Database='Fundacion'
    )
    return connection

def extraccion(sql_query, connection):
    """
    Ejecuta una consulta SQL en la conexión proporcionada y devuelve los resultados en un DataFrame.

    Parameters:
        sql_query (str): La consulta SQL a ejecutar.
        connection: Un objeto de conexión a la base de datos de SQL Server.

    Returns:
        pd.DataFrame: Un DataFrame que contiene los resultados de la consulta.
    
    Excepciones:
        Captura excepciones que puedan ocurrir durante la ejecución de la consulta
        e imprime el error en caso de fallo.
    """
    try:
        df = pd.read_sql(sql_query, connection)
    except Exception as e:
        print(e)
    finally:
        connection.close()  # Asegura que la conexión se cierre después de ejecutar la consulta.
    return df


# Ejemplo de uso
conexion = conexion_sql_server()  # Establece la conexión a la base de datos SQL Server
query = "SELECT * FROM Donaciones"  # Define la consulta SQL para extraer datos de la tabla Donaciones
df = extraccion(query, conexion)  # Ejecuta la consulta y obtiene los datos en un DataFrame
df.head()  # Devuelve las primeras filas del DataFrame


C:\Users\Gaming\AppData\Local\Temp\ipykernel_18856\2623273544.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


,Id_Donacion,Fecha_Donacion,Alta,Baja,CUIT,Frecuencia,Nro_Cuenta,Importe
0,1,2024-02-26,2024-02-26,2024-02-26,20-12342678-9,Mensual,402101,292732.0
1,2,2024-01-12,2024-01-12,2024-01-12,21-23456789-0,Bimestral,403101,326439.0
2,3,2024-06-08,2024-06-08,2024-06-08,22-34567890-1,Anual,403101,276920.0
3,4,2024-02-11,2024-02-11,2024-02-11,23-45678901-2,Mensual,402101,406733.0
4,5,2024-05-29,2024-05-29,2024-05-29,24-56789012-3,Bimestral,403101,352572.0


### Demos un pequeño vistazo a nuestro dataset

In [3]:
# How we see, we have 610 rows and 8 columns 
fs=df.shape
print('Original shape : {}'.format(fs))

#Dropping duplicate values
df = df.drop_duplicates()

# How we see, there were no duplicates
ns= df.shape
print('There were {} duplicates'.format((fs[0])-(ns[0])))
print('New shape : {}'.format(ns))

Original shape : (610, 8)
There were 0 duplicates
New shape : (610, 8)


In [4]:
#More information about the dataset and the data type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id_Donacion     610 non-null    int64  
 1   Fecha_Donacion  610 non-null    object 
 2   Alta            610 non-null    object 
 3   Baja            269 non-null    object 
 4   CUIT            610 non-null    object 
 5   Frecuencia      610 non-null    object 
 6   Nro_Cuenta      610 non-null    int64  
 7   Importe         610 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 38.2+ KB


In [5]:
#check if we have any nulls in the dataset
df.isnull().sum()

Id_Donacion         0
Fecha_Donacion      0
Alta                0
Baja              341
CUIT                0
Frecuencia          0
Nro_Cuenta          0
Importe             0
dtype: int64

In [6]:
#Metrics of the distribution of all numeric features in the dataset
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Id_Donacion,610.0,305.500000,176.236111,1.0,153.25,305.5,457.75,610.0
Nro_Cuenta,610.0,404039.722951,1094.482765,402101.0,403103.00,404100.0,405100.00,409021.0
Importe,610.0,312010.449180,215891.166922,11747.0,119542.50,271959.0,482172.75,1121873.0


In [7]:
#Unique values
df.nunique()

Id_Donacion       610
Fecha_Donacion    450
Alta              104
Baja              217
CUIT              140
Frecuencia          5
Nro_Cuenta          7
Importe           609
dtype: int64

## Conexión con SQLALCHEMY

In [8]:

def configurar_conexion(username: str, password: str, server: str, database: str) -> sqlalchemy.engine.base.Engine:
    """
    Configura una conexión a la base de datos SQL Server utilizando SQLAlchemy.

    Parameters:
        username (str): Nombre de usuario para la conexión a la base de datos.
        password (str): Contraseña para la conexión a la base de datos.
        server (str): Nombre del servidor SQL Server.
        database (str): Nombre de la base de datos a la que se desea acceder.

    Returns:
        engine: Un objeto de motor de conexión creado para acceder a la base de datos.
    
    """
    connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
    engine = sqlalchemy.create_engine(connection_string)
    return engine

def extraccion_datos(query: str, engine: sqlalchemy.engine.base.Engine) -> pd.DataFrame:
    """
    Ejecuta una consulta SQL utilizando el motor de conexión proporcionado y devuelve los resultados en un DataFrame.

    Parameters:
        query (str): La consulta SQL a ejecutar.
        engine: Un objeto de motor de conexión SQLAlchemy.

    Returns:
        pd.DataFrame: Un DataFrame que contiene los resultados de la consulta.
    
    Excepciones:
        Lanza un error si se produce un problema durante la ejecución de la consulta.
    """
    try:
        df = pd.read_sql(query, engine)
    except Exception as e:
        print(e)
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error
    return df

# Ejemplo de uso
username = ''  # Especificar el nombre de usuario
password = ''  # Especificar la contraseña
server = 'DESKTOP-PTELUGB\SQLEXPRESS'
database = 'PizzaDB'

# Configurar la conexión a la base de datos
engine = configurar_conexion(username, password, server, database)

# Definir la consulta SQL para extraer datos de la tabla pizza_sales
query = "SELECT * FROM pizza_sales"

# Ejecutar la consulta y obtener los datos en un DataFrame
data = extraccion_datos(query, engine)

# Mostrar las primeras filas del DataFrame
data.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,1325.0,1325.0,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.0,16.0,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,185.0,185.0,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,2075.0,2075.0,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.0,16.0,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


### Demos un pequeño vistazo a nuestro segundo dataset

In [9]:
# How we see, we have 610 rows and 8 columns 
fs=data.shape
print('Original shape : {}'.format(fs))

#Dropping duplicate values
data = data.drop_duplicates()

# How we see, there were no duplicates
ns= data.shape
print('There were {} duplicates'.format((fs[0])-(ns[0])))
print('New shape : {}'.format(ns))

Original shape : (48620, 12)
There were 0 duplicates
New shape : (48620, 12)


In [10]:
#More information about the dataset and the data type of each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pizza_id           48620 non-null  int64  
 1   order_id           48620 non-null  int64  
 2   pizza_name_id      48620 non-null  object 
 3   quantity           48620 non-null  int64  
 4   order_date         48620 non-null  object 
 5   order_time         48620 non-null  object 
 6   unit_price         48620 non-null  float64
 7   total_price        48620 non-null  float64
 8   pizza_size         48620 non-null  object 
 9   pizza_category     48620 non-null  object 
 10  pizza_ingredients  48620 non-null  object 
 11  pizza_name         48620 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 4.5+ MB


In [11]:
#Metrics of the distribution of all numeric features in the dataset
data.describe().T

,count,mean,std,min,25%,50%,75%,max
pizza_id,48620.0,24310.500000,14035.529381,1.0,12155.75,24310.5,36465.25,48620.0
order_id,48620.0,10701.479761,6180.119770,1.0,5337.00,10682.5,16100.00,21350.0
quantity,48620.0,1.019622,0.143077,1.0,1.00,1.0,1.00,4.0
unit_price,48620.0,955.201851,880.919175,11.0,125.00,975.0,2025.00,3595.0
total_price,48620.0,940.389099,880.498462,11.0,125.00,415.0,1795.00,6225.0


In [12]:
#Unique values
data.nunique()

pizza_id             48620
order_id             21350
pizza_name_id           91
quantity                 4
order_date             358
order_time           16382
unit_price              25
total_price             56
pizza_size               5
pizza_category           4
pizza_ingredients       32
pizza_name              32
dtype: int64

Además de establecer la conexión con SQL Server, revisar la estructura y el tipo de datos fue un paso clave para entender cómo están organizados y asegurarnos de que sean adecuados para el análisis o procesamiento que queremos realizar. Este análisis inicial no solo nos ayuda a prevenir errores más adelante, sino que también permite optimizar el trabajo con los datos, garantizando que las herramientas y consultas que implementemos sean las correctas para alcanzar nuestros objetivos.